In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
matplotlib.rcParams['font.size'] = 12

In [ ]:
class Gridworld:
  def __init__(self, size=6, traps=[], alpha=0):
    self.size = size
    self.traps = traps
    self.alpha = alpha
    self.start = (0, 0)
    self.goal = (size-1, size-1)

    self.states = [(x, y) for x in range(size) for y in range(size)]
    self.actions = [(-1, 0), (0, -1), (1, 0), (0, 1)]
    
    self.policy = {}
    for s in self.states:
      self.policy[(s, (-1, 0))] = 0
      self.policy[(s, (0, -1))] = 0
      self.policy[(s, (1, 0))] = 1/2
      self.policy[(s, (0, 1))] = 1/2

    self.value = {}
    for s in self.states:
      self.value[s] = 0
   
  def move(self, s, a):
    if s == self.goal:
      return [(1, 0, s)]      # Probability, Reward, Next state

    s_new = (s[0] + a[0], s[1] + a[1])

    if s_new not in self.states:
      return [(1, 0, s)]      # Probability, Reward, Next state

    if s_new in self.traps:
      # Probability, Reward, Next state
      return [(self.alpha, -1, self.start), (1-self.alpha, -1, self.goal)]

    return [(1, -1, s_new)]   # Probability, Reward, Next state

In [ ]:
def show_values(world, subplot=None, title='Values'):
  if not subplot:
    fig = plt.figure(figsize=(world.size*0.8, world.size*0.8))
    subplot = fig.add_subplot(1, 1, 1)

  result = np.zeros([world.size, world.size])
  for (x, y) in world.states:
    if (x, y) in world.traps:
      result[y][x] = None
    else:
      result[y][x]  = world.value[(x, y)]
  sns.heatmap(result, square=True, cbar=False,
              annot=True, fmt='3.1f', cmap='coolwarm',
              ax=subplot).set_title(title)